# Assignment 2

In [ ]:
# Imports
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn import linear_model

## Hemnet dataset
The Hemnet data contains information about the selling prices of villas in Landvetter that were sold in the
previous 12 months.

### 1
Find a linear regression model that relates the living area to the selling price. If in doing so, you
performed any data cleaning step(s), describe what you did and explain why.

### 2
What are the values of the slope and intercept of the regression line?

### 3
Use this model to predict the selling prices of houses which have living area 100 m2, 150 m2, and
200 m2.

### 4
Draw a residual plot. Discuss some potential strategies for improving the model.

## Iris dataset

In [ ]:
from sklearn.datasets import load_iris

### 1
Use a confusion matrix to evaluate the use of logistic regression to classify the Iris data set.

### 2
Use k-nearest neighbors (k-NN) to classify the Iris data set with some different values for k, and
with uniform and distance-based weights. What will happen when k grows larger for the different
cases? Why?

### 3
Compare the classification models for the Iris data set that are generated by k-nearest neighbors
(for the different settings from question 2) and by logistic regression. Calculate confusion matrices
for these models and discuss the performance of the various models.